In [1]:
from collections import OrderedDict
import torch.nn.functional as F
import torch.nn as nn
import torch, os
import numpy as np, six, re
from modeling import *
from easydict import EasyDict as edict
from pprint import pprint

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [2]:
!wget -P models/ https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -d models/ models/uncased_L-12_H-768_A-12.zip 
!rm -rf models/uncased_L-12_H-768_A-12.zip

--2020-05-04 18:49:14--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘models/uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  6.30MB/s    in 71s     

2020-05-04 18:50:26 (5.47 MB/s) - ‘models/uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  models/uncased_L-12_H-768_A-12.zip
   creating: models/uncased_L-12_H-768_A-12/
  inflating: models/uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: models/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: models/uncased_L-12_H-768_A-12/vocab.txt  
  inflating: models/uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: models/uncased_L-12_H-7

In [3]:
tf_path = os.path.abspath('models/uncased_L-12_H-768_A-12/bert_model.ckpt')
config = BertConfig.from_json_file('models/uncased_L-12_H-768_A-12/bert_config.json')
with open('models/uncased_L-12_H-768_A-12/bert_config.json', 'r') as f:
    _config = edict(json.load(f))

tf_vars = tf.train.list_variables(tf_path)
def list_tf_var_by_scope(scope):
    return [i for i in tf_vars if scope in i[0]]

def load_tf_var_as_numpy(name):
    return tf.train.load_variable(tf_path, name)

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('models/uncased_L-12_H-768_A-12/bert_model.ckpt.meta')
saver.restore(sess, 'models/uncased_L-12_H-768_A-12/bert_model.ckpt')

graph = tf.get_default_graph()
node_name = [i.name for i in graph.as_graph_def().node]

INFO:tensorflow:Restoring parameters from models/uncased_L-12_H-768_A-12/bert_model.ckpt


In [4]:
def reshape_to_matrix(input_tensor):
    ndims = len(input_tensor.shape)
    if ndims < 2:
        raise ValueError("Input tensor must have at least rank 2. Shape = %s" %(input_tensor.shape))
    if ndims == 2:
        return input_tensor
    
    width = input_tensor.shape[-1]
    output_tensor = torch.reshape(input_tensor, [-1, width])
    return output_tensor

def reshape_from_matrix(output_tensor, orig_shape_list):
    if len(orig_shape_list) == 2:
        return output_tensor
    output_shape = list(output_tensor.shape)
    
    orig_dims = orig_shape_list[0:-1]
    width = output_shape[-1]
    return torch.reshape(output_tensor, orig_dims + [width])

def assert_rank(tensor, expected_rank, name=None):
    
    if name is None:
        name = tensor.name
        
    expected_rank_dict = {}
    if isinstance(expected_rank, six.integer_types):
        expected_rank_dict[expected_rank] = True
    else:
        for x in expected_rank:
            expected_rank_dict[x] = True
    actual_rank = len(tensor.shape)
    if actual_rank not in expected_rank_dict:
        raise ValueError("For the tensor `%s`, the actual rank `%d` (shape = %s) is not equal to the expected rank `%s`" %
                        (name, actual_rank, str(tensor.shape), str(expected_rank)))
        
def get_shape_list(tensor, expected_rank=None, name=None):
    if name is None:
        name = tensor.name
        
    if expected_rank is not None:
        assert_rank(tensor, expected_rank, name)
        
    shape = list(tensor.shape)
    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        if dim is None:
            non_static_indexes.append(index)

    if not non_static_indexes:
        return shape

    dyn_shape = torch.shape(tensor)
    for index in non_static_indexes:
        shape[index] = dyn_shape[index]
    return shape
    
def gelu(x):
    cdf = 0.5*(1.0 + torch.tanh(
        np.sqrt(2/np.pi) * (x + 0.044715 * torch.pow(x, 3))))
    return x * cdf

def create_attention_mask_from_input_mask(from_tensor, to_mask):
    from_shape = get_shape_list(from_tensor, expected_rank=[2,3])
    batch_size = from_shape[0]
    from_seq_length = from_shape[1]

    to_shape = get_shape_list(to_mask, expected_rank=2)
    to_seq_length = to_shape[1]
    
    to_mask = torch.reshape(to_mask, [batch_size, 1, to_seq_length]).type(torch.FloatTensor)    
    broadcast_ones = torch.ones([batch_size, from_seq_length, 1]).type(torch.FloatTensor)
    mask = broadcast_ones * to_mask
    return mask

In [5]:
class AttentionSelf(nn.Module):
    
    def __init__(self, 
                 config,
                 do_return_2d_tensor=True, 
                 batch_size=1,
                 num_attention_heads=12, 
                 size_per_head=64,
                 from_seq_length=128,
                 to_seq_length=128):
        
        super(AttentionSelf, self).__init__()
        self.query = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.key = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.value = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        
        self.do_return_2d_tensor = do_return_2d_tensor
        self.batch_size = batch_size
        self.num_attention_heads = num_attention_heads
        self.size_per_head = size_per_head
        self.from_seq_length = from_seq_length
        self.to_seq_length = to_seq_length
        
    def transpose_for_scores(self, 
                            input_tensor, batch_size=1, seq_length=128, 
                             num_attention_heads=12, width=64):
        output_tensor = torch.reshape(input_tensor, 
                    [batch_size, seq_length, num_attention_heads, width])
        output_tensor = output_tensor.permute(0, 2, 1, 3)
        return output_tensor
        
    def forward(self, from_tensor, to_tensor, attention_mask):
        
        query_layer = self.query(from_tensor)
        key_layer = self.key(to_tensor)
        value_layer = self.value(to_tensor)
        
        query_layer = self.transpose_for_scores(query_layer, 
            self.batch_size, self.from_seq_length, self.num_attention_heads, self.size_per_head)
        key_layer = self.transpose_for_scores(key_layer, 
            self.batch_size, self.to_seq_length, self.num_attention_heads, self.size_per_head)
        
        attention_scores = torch.matmul(query_layer, torch.transpose(key_layer, -1, -2))
        attention_scores /= math.sqrt(self.size_per_head)
        
        if attention_mask is not None:
            attention_mask = torch.unsqueeze(attention_mask, dim=1)
            adder = (1.0 - attention_mask) * -10000.0
            attention_scores += adder
            
        attention_probs = F.softmax(attention_scores, -1)
        attention_probs = self.dropout(attention_probs)
        
        value_layer = torch.reshape(value_layer, 
            [self.batch_size, self.to_seq_length, self.num_attention_heads, self.size_per_head])
        value_layer = value_layer.permute(0, 2, 1, 3)
        
        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3)
        if self.do_return_2d_tensor:
            context_layer = torch.reshape(context_layer, 
            [self.batch_size * self.from_seq_length, self.num_attention_heads * self.size_per_head])
        else:
            context_layer = torch.reshape(context_layer, 
            [self.batch_size, self.from_seq_length, self.num_attention_heads * self.size_per_head])
        
        return context_layer
    
    
class AttentionOutput(nn.Module):
    
    def __init__(self, config):
        super(AttentionOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.LayerNorm = nn.LayerNorm(config.hidden_size)
        
    
    def forward(self, attention_output, layer_input):
        attention_output = self.dense(attention_output)
        attention_output = self.dropout(attention_output)
        attention_output = self.LayerNorm(attention_output+layer_input)
        return attention_output
    
    
class Attention(nn.Module):
    
    def __init__(self, config):
        super(Attention, self).__init__()
        self.self = AttentionSelf(config)
        self.output = AttentionOutput(config)
        
    def forward(self, from_tensor, to_tensor, attention_mask):
        attention_heads = []
        attention_head = self.self(
                                        from_tensor=from_tensor, 
                                        to_tensor=to_tensor, 
                                        attention_mask=attention_mask)
        attention_heads.append(attention_head)
        attention_output = None
        if len(attention_heads) == 1:
            attention_output = attention_heads[0]
        else:
            attention_output = torch.cat(attention_heads, dim=-1)
            
        attention_output = self.output(attention_output, from_tensor)
        return attention_output
    
    
class Intermediate(nn.Module):
    
    def __init__(self, config):
        super(Intermediate, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.intermediate_act_fn = gelu
        
    def forward(self, activation_output):
        return self.intermediate_act_fn(self.dense(activation_output))
    
    
class Output(nn.Module):
    
    def __init__(self, config):
        super(Output, self).__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.LayerNorm = nn.LayerNorm(config.hidden_size)
        
    def forward(self, intermediate_output, attention_output):
        layer_output = self.dense(intermediate_output)
        layer_output = self.dropout(layer_output)
        layer_output = self.LayerNorm(layer_output + attention_output)
        return layer_output
    
    
class TransformerBlock(nn.Module):
    
    def __init__(self, config):
        super(TransformerBlock, self).__init__()
        self.attention = Attention(config)
        self.intermediate = Intermediate(config)
        self.output = Output(config)
        
    def forward(self, layer_input, attention_mask):
        attention_output = self.attention(from_tensor=layer_input, 
                                          to_tensor=layer_input, 
                                          attention_mask=attention_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

In [6]:
class Embeddings(nn.Module):
    
    def __init__(self, config):
        super(Embeddings, self).__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size)
    
    def forward(self, input_ids=None, token_type_ids=None, position_ids=None):
        word_embeddings = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        position_embeddings = self.position_embeddings(position_ids)
        
        return self.LayerNorm(word_embeddings+token_type_embeddings+position_embeddings)
    
    
class Encoder(nn.Module):
    def __init__(self, config, do_return_all_layers=True):
        super(Encoder, self).__init__()
        self.layer = nn.ModuleList([TransformerBlock(config) for _ in range(config.num_hidden_layers)])
        self.do_return_all_layers = do_return_all_layers
        
    def forward(self, input_tensor, attention_mask):
        
        input_shape = get_shape_list(input_tensor, expected_rank=3)
        prev_output = reshape_to_matrix(input_tensor)
        
        all_layer_outputs = []
        for layer in self.layer:
            layer_input = prev_output
            layer_output = layer(layer_input, attention_mask)
            prev_output = layer_output
            all_layer_outputs.append(layer_output)
            
        if self.do_return_all_layers:
            final_outputs = []
            for layer_output in all_layer_outputs:
                final_output = reshape_from_matrix(layer_output, input_shape)
                final_outputs.append(final_output)
            return final_outputs
        else:
            final_output = reshape_from_matrix(prev_output, input_shape)
            return final_output
        
        
class Pooler(nn.Module):
    
    def __init__(self, config):
        super(Pooler, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        
    def forward(self, sequence_output):
        first_token_tensor = torch.squeeze(sequence_output[:, 0:1, :], dim=0)
        pooled_output = torch.tanh(self.dense(first_token_tensor))
        return pooled_output
        
        
class BertModel(nn.Module):
    
    def __init__(self, config):
        super(BertModel, self).__init__()
        self.embeddings = Embeddings(config)
        self.encoder = Encoder(config)
        self.pooler = Pooler(config)
        
    def forward(self, input_ids, input_mask=None, token_type_ids=None):
        
        input_shape = get_shape_list(input_ids, expected_rank=2)
        batch_size = input_shape[0]
        seq_length = input_shape[1]
        
        if input_mask is None:
            input_mask = torch.ones([batch_size, seq_length]).type(torch.LongTensor)
        
        if token_type_ids is None:
            token_type_ids = torch.zeros([batch_size, seq_length]).type(torch.LongTensor)
        
        # embeddings scope
        embedding_output = self.embeddings(input_ids, token_type_ids, position_ids)
        
        # encoder scope
        attention_mask = create_attention_mask_from_input_mask(input_mask, input_ids) 
        
        all_encoder_layers = self.encoder(embedding_output, attention_mask)
        sequence_output = all_encoder_layers[-1]
        
        # pooler scope
        pooled_output = self.pooler(sequence_output)
        return pooled_output

### Run forward pass

In [7]:
def layer_tf2torch(text=None, model_name='bert'):
    if 'LayerNorm' in text:
        if 'weight' in text:
            text = text.replace('weight', 'gamma')
        elif 'bias' in text:
            text = text.replace('bias', 'beta')
    else:
        text = text.replace('weight', 'kernel')
    layer_idx = re.findall('layer.\d+', text)
    if len(layer_idx):
        layer_idx = layer_idx[0]
        new_layer_idx = layer_idx.replace('.', '_')
        text = text.replace(layer_idx, new_layer_idx)
    if '_embeddings' in text:
        text = text.replace('kernel', '')
    return '/'.join([model_name, text.replace('.', '/')]).strip('/')

def load_bert_weights(tf_path, torch_model, model_name):
    
    def _import_weights(tf_path, name):
        arr = tf.train.load_variable(tf_path, name)
        if 'kernel' in name:
            return arr.transpose()
        return arr
    
    loaded_state_dict = OrderedDict()
    for i in list(torch_model.state_dict().keys()):
        torch_weight = _import_weights(tf_path, layer_tf2torch(i, model_name))
        loaded_state_dict[i] = torch.tensor(torch_weight)
    return loaded_state_dict

In [8]:
bertmodel = BertModel(config)
bertmodel.eval()
loaded_state_dict = load_bert_weights(tf_path, bertmodel, 'bert')
bertmodel.load_state_dict(loaded_state_dict)

<All keys matched successfully>

In [10]:
input_ids = torch.tensor(np.expand_dims(np.arange(128), 0), dtype=torch.long)

input_shape = input_ids.size()
seq_length = input_shape[1]
position_ids = torch.arange(seq_length, dtype=torch.long)
token_type_ids = torch.zeros(input_shape, dtype=torch.long)

out = bertmodel(input_ids)
print(out)

tensor([[-1.9556e-01, -3.8529e-01, -1.2395e-01,  6.4992e-01,  3.9835e-01,
         -2.6433e-01,  4.4037e-01,  1.1338e-01, -5.4311e-01, -4.8537e-01,
         -4.6098e-01,  9.5167e-01,  9.2152e-01, -4.9869e-01,  9.0811e-01,
          3.6789e-02, -6.3752e-01,  2.1472e-01,  3.2824e-01,  5.0361e-01,
          8.3490e-01,  9.9977e-01, -5.0796e-02,  1.4625e-01,  1.5398e-01,
          9.9142e-01, -7.9022e-01,  8.9986e-01,  8.4503e-01,  3.2737e-01,
         -9.3161e-02,  1.0036e-01, -9.8784e-01, -2.3155e-01, -8.6401e-01,
         -9.0584e-01,  1.9727e-01, -2.7658e-01,  2.8379e-03,  8.9015e-02,
         -8.9106e-01,  2.2820e-01,  8.8577e-01,  3.9246e-01,  8.1413e-01,
          9.0167e-02, -9.9637e-01,  2.1161e-01, -6.1490e-01,  5.2413e-01,
          5.7233e-01,  9.6379e-01,  2.2309e-02,  2.5973e-01,  2.5539e-01,
         -3.1383e-01,  7.0062e-02,  9.4177e-02,  2.1852e-02, -3.6695e-01,
         -4.2327e-01,  4.3701e-01, -6.9330e-01, -2.9266e-01,  8.6426e-01,
         -4.9095e-01, -2.5013e-01, -1.